In [1]:
import numpy as np
import pickle
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import pickle
%matplotlib inline

with open('obj_pts.p', 'rb') as f:
    objpoints = pickle.load(f)
with open('img_pts.p', 'rb') as f:
    imgpoints =pickle.load(f)
with open("M.p", "rb") as f:
    M = pickle.load(f)
with open("inverseM.p", "rb") as f:
    inverseM = pickle.load(f)

  warn(skimage_deprecation('The `skimage.filter` module has been renamed '



In [2]:
#get X or Y gradient via Sobel Operator.
def get_abs_Sobel_grad(gray_img, sobel_kernel=5, direction = 'X'):
    if direction == 'X':
        sobelx=cv2.Sobel(gray_img, cv2.CV_64F, 1, 0,sobel_kernel)
        return np.absolute(sobelx)
    elif direction =='Y':
        sobely=cv2.Sobel(gray_img, cv2.CV_64F, 0, 1,sobel_kernel)
        return np.absolute(sobely)
    else:
        print('Please set gradient direction parameter: X or Y?')
        return gray_img

#Scale the gradient image.
def get_scaled_Sobel(abs_sobel_img):
    return np.uint8(255*abs_sobel_img/np.max(abs_sobel_img))

#get binary image of gradient.
def binary_Sobel(scaled_img, threshold=(20, 100)):
    binary_sobel = np.zeros_like(scaled_img)
    binary_sobel[(scaled_img>=threshold[0])&(scaled_img<=threshold[1])]=1
    return binary_sobel

#get binary image of gradient's magnitude
def binary_Magnitude(abs_sobelx, abs_sobely, threshold=(20, 100)):    
    sobelxy = np.sqrt(np.square(abs_sobelx)+np.square(abs_sobely)) 
    scaled_sobelxy = np.uint8(255*sobelxy/np.max(sobelxy))
    binary_sobelxy = np.zeros_like(scaled_sobelxy)
    binary_sobelxy[(scaled_sobelxy >= threshold[0]) & (scaled_sobelxy <= threshold[1])] = 1
    return binary_sobelxy

#get binary image of direction of gradient
def binary_Dir_Grad(abs_sobelx, abs_sobely, arc_threshold=(0.7, 1.3)):
    dir_grad = np.arctan2(abs_sobely, abs_sobelx)
    binary_dir_grad = np.zeros_like(dir_grad)
    binary_dir_grad[(dir_grad>=arc_threshold[0]) & (dir_grad<=arc_threshold[1])] = 1
    return binary_dir_grad
                    
def combine_Binary_Grad(binary_sbx, binary_sby, binary_mag, binary_dir_grad):
    binary_Comb_Grad = np.zeros_like(binary_sbx)
    binary_Comb_Grad[((binary_sbx==1)&(binary_sby==1))|((binary_mag==1)&(binary_dir_grad==1))] = 1
    return binary_Comb_Grad

In [3]:
def get_HLS_channel(undist_img, channel='S', threshold=(0,255)):
    HLS_img = cv2.cvtColor(undist_img, cv2.COLOR_BGR2HLS)
    if channel=='H':
        H = HLS_img[:,:,0]
        binary_H = np.zeros_like(H)
        binary_H[(H>=threshold[0])&(H<=threshold[1])]=1
        return binary_H
    elif channel=='L':
        L = HLS_img[:,:,1]
        binary_L = np.zeros_like(L)
        binary_L[(L>=threshold[0])&(L<=threshold[1])]=1
        return binary_L
    elif channel=='S':
        S = HLS_img[:,:,2]
        binary_S = np.zeros_like(S)
        binary_S[(S>=threshold[0])&(S<=threshold[1])]=1
        return binary_S
    else:
        print('Please choose one channel parameter of H, L or S?')
        return undist_img
def combine_HS(binary_H, binary_S):
    binary_HS = np.zeros_like(binary_H)
    binary_HS[(binary_H ==1)&(binary_S ==1)] = 1
    return binary_HS

In [4]:
def combine_Binary(binary_color, binary_grad):
    color_grad_comb = np.zeros_like(binary_grad)
    color_grad_comb[(binary_color==1) | (binary_grad==1)]=1
    return color_grad_comb

In [6]:
#This function calculates points of lanes for fitting curve.
def calc_Lane_Points(binary_Combine):
    mid_height = np.int(binary_Combine.shape[0]/2)
    histogram = np.sum(binary_Combine[mid_height:,:],axis=0)
    mid_width = np.int(histogram.shape[0]/2)
    print("The middle point of image:{}".format(mid_width))
    leftx_base = np.argmax(histogram[:mid_width])
    rightx_base = np.argmax(histogram[mid_width:]) + mid_width
    print("Max value at x-{} left part.".format(leftx_base))
    print("Max value at x-{} right part.".format(rightx_base))
    nonzero = np.nonzero(binary_Combine)
    nonzerox = nonzero[1]
    nonzeroy = nonzero[0]
    nwindows = 9
    height = binary_Combine.shape[0]
    win_height = np.int(binary_Combine.shape[0]/nwindows)
    margin = 100
    minpixel = 50
    #These two empty lists store indexes of nonzero pixels in nonzerox and nonzeroy.
    left_lane_index = []
    right_lane_index = []
    leftx_current = leftx_base
    rightx_current = rightx_base
    for window in range(nwindows):
        win_y_low = height - (window+1)*win_height
        win_y_high = height - window*win_height
        win_leftx_left = leftx_current - margin
        win_leftx_right = leftx_current + margin
        win_rightx_left = rightx_current - margin
        win_rightx_right = rightx_current + margin
        good_left_ind=((nonzeroy>=win_y_low)&(nonzeroy<win_y_high)&(nonzerox>=win_leftx_left)&(nonzerox<win_leftx_right)).nonzero()[0]
        good_right_ind=((nonzeroy>=win_y_low)&(nonzeroy<win_y_high)&(nonzerox>=win_rightx_left)&(nonzerox<win_rightx_right)).nonzero()[0]
        left_lane_index.append(good_left_ind)
        right_lane_index.append(good_right_ind)
        if len(good_left_ind) > minpixel:
            leftx_current = np.int(np.mean(nonzerox[good_left_ind]))
        if len(good_right_ind) > minpixel:
            rightx_current = np.int(np.mean(nonzerox[good_right_ind]))
    left_lane_index = np.concatenate(left_lane_index)
    right_lane_index = np.concatenate(right_lane_index)

    leftx = nonzerox[left_lane_index]
    lefty = nonzeroy[left_lane_index] 
    rightx = nonzerox[right_lane_index]
    righty = nonzeroy[right_lane_index]
    return leftx, lefty, rightx, righty

#This function gets left and right lanes via A*y*y+B*y+C which returns A, B and C coefficients.
def fit_Lanes(leftx, lefty, rightx, righty):
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    return left_fit, right_fit

#This function calculates the whole lanes using fit coefficients A, B, C
def calc_X(binary_Combine, left_fit, right_fit):
    ploty = np.linspace(0, binary_Combine.shape[0]-1, binary_Combine.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    return left_fitx, right_fitx

In [7]:
def process_image(img, First_Frame = True):
    #undist = undistortImage(img, objpoints, imgpoints)
    warped_img = cv2.warpPerspective(img, M, (1280, 720))
    gray_warped = cv2.cvtColor(warped_img, cv2.COLOR_RGB2GRAY)
    warped_sobelx = get_abs_Sobel_grad(gray_warped, sobel_kernel=5, direction = 'X')
    warped_sobely = get_abs_Sobel_grad(gray_warped, sobel_kernel=5, direction = 'Y')
    scaled_warped_sobelx = get_scaled_Sobel(warped_sobelx)
    scaled_warped_sobely = get_scaled_Sobel(warped_sobely)

    binary_warped_sobelx = binary_Sobel(scaled_warped_sobelx, threshold=(20, 100))
    binary_warped_sobely = binary_Sobel(scaled_warped_sobely, threshold=(20, 100))
    #binary_mag = binary_Magnitude(warped_sobelx, warped_sobely, threshold=(28, 141))
    #binary_dir_grad = binary_Dir_Grad(warped_sobelx, warped_sobely, arc_threshold=(1.0, np.pi/2))

    #binary_grad = combine_Binary_Grad(binary_warped_sobelx, binary_warped_sobely, binary_mag, binary_dir_grad)
    
    binary_S = get_HLS_channel(warped_img, threshold=(170,255))
    binary_H = get_HLS_channel(warped_img, 'H',threshold=(25,40))
    #binary_L = get_HLS_channel(warped_img, 'L',threshold=(24,200))
    #binary_HS = combine_HS(binary_H, binary_S)
    
    binary_Combine = combine_Binary(binary_S, binary_warped_sobelx)
    
    if First_Frame:
        #detected pixels on lanes
        leftx, lefty, rightx, righty = calc_Lane_Points(binary_Combine)
        First_Frame = False
    else:
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        margin = 100
        left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
        left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
        left_fit[1]*nonzeroy + left_fit[2] + margin))) 

        right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
        right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
        right_fit[1]*nonzeroy + right_fit[2] + margin)))  

        # Again, extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]
    
    # Fit a second order polynomial to each.calculates coefficients
    left_fit, right_fit = fit_Lanes(leftx, lefty, rightx, righty)
    #calculate x position of lanes via coefficients
    left_fitx, right_fitx = calc_X(binary_Combine, left_fit, right_fit)
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_Combine).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    ploty = np.linspace(0, binary_Combine.shape[0]-1, binary_Combine.shape[0])
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, inverseM, (1280, 720)) 
    # Combine the result with the original image
    result = cv2.addWeighted(img, 1, newwarp, 0.3, 0)
    return result

In [8]:
video_output = "../output_images/project_video_output.mp4"
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip1 = VideoFileClip("../project_video.mp4").subclip(0,5)
clip1 = VideoFileClip("../project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(video_output, audio=False)

The middle point of image:640
Max value at x-343 left part.
Max value at x-1036 right part.
[MoviePy] >>>> Building video ../output_images/project_video_output.mp4
[MoviePy] Writing video ../output_images/project_video_output.mp4


  0%|                                                 | 0/1261 [00:00<?, ?it/s]

The middle point of image:640
Max value at x-343 left part.
Max value at x-1036 right part.


  0%|                                         | 1/1261 [00:00<07:08,  2.94it/s]

The middle point of image:640
Max value at x-343 left part.
Max value at x-1038 right part.


...

The middle point of image:640
Max value at x-357 left part.
Max value at x-1066 right part.


100%|█████████████████████████████████████▉| 1259/1261 [05:51<00:00,  4.98it/s]

The middle point of image:640
Max value at x-362 left part.
Max value at x-1067 right part.


100%|█████████████████████████████████████▉| 1260/1261 [05:51<00:00,  3.47it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ../output_images/project_video_output.mp4 

Wall time: 5min 54s
